In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M:%S')
usecols = ["id", "date", "user", "pc", "activity"]
logon_df = pd.read_csv('../data/r2/logon_test.csv', parse_dates=['date'], date_parser=dateparse, usecols=usecols)
logon_df = logon_df.set_index('date').sort_index(axis=0)

In [ ]:
logon_df.info(memory_usage='deep')

In [ ]:
logon_df.head(10)

1. User who did not previously use removable drives or work after
hours begins logging in after hours, using a removable drive, and
uploading data to wikileaks.org. Leaves the organization shortly
thereafter.

In [ ]:
# create a field for the hour of the day
logon_df['hour'] = logon_df.index.hour
logon_df.head()

In [ ]:
# group the data by user, hour and activity and resample/sum on a daily basis
# results in a df that shows the count of logons/logoffs for each user, hour for each day
logon_resampled_df = logon_df.groupby(['user','hour','activity'], sort=False).resample('D').count()[['id']]
logon_resampled_df = logon_resampled_df.rename(columns={'id': 'daily_count'})
logon_resampled_df.head(30)

In [ ]:
# removed the leveled index that was set by the group_by
logon_resampled_df = logon_resampled_df.reset_index()
logon_resampled_df = logon_resampled_df.set_index('date').sort_index(axis=0)
#logon_resampled_df.head(100)

In [ ]:
periods = 2

# calculate the mean number of logins for each hour over the last n days
user_hour_df = logon_resampled_df.groupby(['user','hour','activity']).rolling(periods, min_periods=1).mean()
user_hour_df.head(30)

# TODO - need to find a way to exclude the current day from a rolling window
# shift() won't work here since shift moves the entire series independent of the aggregations

2. User begins surfing job websites and soliciting employment from a
competitor. Before leaving the company, they use a thumb drive (at
markedly higher rates than their previous activity) to steal data.

3. System administrator becomes disgruntled. Downloads a keylogger and
uses a thumb drive to transfer it to his supervisor's machine. The
next day, he uses the collected keylogs to log in as his supervisor
and send out an alarming mass email, causing panic in the
organization. He leaves the organization immediately.

4. A user logs into another user's machine and searches for
interesting files, emailing to their home email. This behavior occurs
more and more frequently over a 3 month period.

5. A member of a group decimated by layoffs uploads documents to
Dropbox, planning to use them for personal gain.